# Gold and Silver Prices

The visualization illustrates the current USD price of Gold and Silver.

In [1]:
# Import notebook libraries and dependencies
import os
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import alpha_vantage as av
from alpha_vantage.foreignexchange import ForeignExchange

In [2]:
#Set API keys
av_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

In [3]:
# Retreive Gold/USD exchange rate 
fe = ForeignExchange(key=av_api_key)
data, _ = fe.get_currency_exchange_rate(from_currency='XAU',to_currency='USD')
data['5. Exchange Rate']=pd.to_numeric(data['5. Exchange Rate'],errors='coerce')
gold_price = round(data['5. Exchange Rate'],2)
print(f"The Price of Gold is: ${gold_price}")

The Price of Gold is: $1939.1


In [4]:
# Retreive Silver/USD exchange rate 
fe = ForeignExchange(key=av_api_key)
data, _ = fe.get_currency_exchange_rate(from_currency='XAG',to_currency='USD')
data['5. Exchange Rate']=pd.to_numeric(data['5. Exchange Rate'],errors='coerce')
silver_price = round(data['5. Exchange Rate'],2)
print(f"The Price of Silver is: ${silver_price}")

The Price of Silver is: $26.65


In [5]:
# Calculate Gold/Silver Ratio
goldsilver_ratio = round(gold_price/silver_price,2)
print(f"The Gold/Silver ratio is: {goldsilver_ratio}")

The Gold/Silver ratio is: 72.76
